In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf

from keras import layers
from keras import models
from keras import optimizers
from keras import preprocessing
from keras.applications import ResNet101
from keras.datasets import mnist
from keras.datasets import imdb
from keras.layers import Flatten, Dense, Embedding
from keras.layers import SimpleRNN, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from konlpy.tag import Twitter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



import math, sys
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

#tf.compat.v1.disable_eager_execution()

In [ ]:
with tf.compat.v1.Session() as ses:
    # Build a graph.
    a = tf.constant(5.0)
    b = tf.constant(6.0)
    c = a * b

    # Evaluate the tensor `c`.
    print(ses.run(c))
    
    a = tf.constant(120,name = "a")  #상수 생성
    b = tf.constant(130,name = "b")
    c = tf.constant(140,name = "c")
    v = tf.Variable(0, name = "v")   #변수 생성

    calc_op = a + b + c                  #수식 입력
    assign_op = tf.assign(v, calc_op)#계싼 결과와 변수연결

    sess = tf.Session()              #세션 수립, 시작
    sess.run(assign_op)              #연산 실행

    print(sess.run(v))               #v에 담긴값 출력
    


In [ ]:
with tf.compat.v1.Session() as ses:    
    a = tf.placeholder(tf.int32, [3])
    print(a)
    b = tf.constant(2)
    print(b)
    x2_op = a * b
    
    r1 = ses.run(x2_op, feed_dict = {a:[1,2,3]})
    print(r1)
    
    r2 = ses.run(x2_op, feed_dict = {a:[10,20,30]})
    print(r2)


In [ ]:
import numpy as np
import pandas as pd
xor_data = [[0,0],[0,1],[1,0],[1,1]]
xor_label = [[0],[1],[1],[0]]

x = tf.placeholder(tf.float32, [None,2])#데이터
y = tf.placeholder(tf.float32, [None,1])#라벨

w = tf.Variable(tf.zeros([2,1]))#가중치 초기화
b = tf.Variable(tf.zeros([1])) # 바이어스 초기화

a = tf.nn.softmax(tf.matmul(x, w) + b) #소프트맥스 회귀함수 정의
cross_entropy = -tf.reduce_sum(a * tf.log(a)) # 크로스엔트로피 정의
optimizer = tf.train.GradientDescentOptimizer(0.01) #옵티마이저 정의
train = optimizer.minimize(cross_entropy) #학습 시작

predict = tf.equal(tf.argmax(a,1), tf.argmax(y,1)) # 정답률 계산 수식
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

sess = tf.Session()  # 실행
sess.run(tf.global_variables_initializer())
fd = {x: xor_data, y : xor_label}

for i in range(10000):
    sess.run(train, feed_dict=fd)#훈련
    if i % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)#w, b 재조정
        acc = sess.run(accuracy, feed_dict={x: xor_data, y: xor_label})
        print(i, "번 수행. 정답률 =", acc)
acc = sess.run(accuracy, feed_dict={x: xor_data, y: xor_label})#테스팅
print("최종 정답률 =", acc)
pre = sess.run(tf.argmax(a, 1), feed_dict={x: [[1,0]]})#학습 확인
print(pre)

In [ ]:
tf.compat.v1.disable_eager_execution()
csv = pd.read_csv("../input/bmicsv/bmi.csv")

csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
#one-hot 레이블
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

x  = tf.placeholder(tf.float32, [None, 2]) 
y_ = tf.placeholder(tf.float32, [None, 3]) 

W = tf.Variable(tf.zeros([2, 3])); #가중치
b = tf.Variable(tf.zeros([3])); #바이어스 
#y = wx+b. 실제 값과 예측값의 분산을 계산할 회귀함수 지정
y = tf.nn.softmax(tf.matmul(x, W) + b)

#손실을 최소화하기 위한 최적화
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

#정답
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
#정답률
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화하기

for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]#무작위 위치의 100줄 추출
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)#학습
    #테스팅
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)

acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("정답률 =", acc)

In [ ]:

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))

In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:


(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images - test_images.astype('float32') /  255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_images, train_labels, epochs = 5, batch_size = 64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

In [ ]:
import numpy as np
print(test_images.shape)
pre = model.predict(test_images)
print(test_labels[np.argmax(pre[4])])

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

conv2D : convolution layer
param1 : filter's dim.
param2 : filterSize, stribe, padding(zero padding = 'same', else = 'valid')

MaxPooling2D : featureMap's size down


In [ ]:
model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:

original_dataset_dir = '../input/dataset/datasets/cats_and_dogs/train/'
base_dir = '../input/dataset/datasets/cats_and_dogs_small/'
validation_dir = base_dir +'validation/'
test_dir = base_dir + 'test/'
train_dir = base_dir + 'train/'
train_cats_dir = train_dir +'cats/'
train_dogs_dir = train_dir + 'dogs/'

vaildation_cats_dir = validation_dir + 'cats/'
vaildation_dogs_dir = validation_dir + 'dogs/'

test_cats_dir = test_dir + 'cats'
test_dogs_dir = test_dir + 'dogs'

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size = 20, class_mode = 'binary')   #class_model => 결과의 개수에 따라 binary, classify 로나뉨
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size = (150,150), batch_size = 20, class_mode = 'binary')

from keras.preprocessing import image

img_path = 'cat1.jpg'
img = image.load_img(img_path, target_size=(150,150,))

x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0 
for batch in datagen.flow(x, batch_size = 1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i+= 1
    if i % 4 == 0: break
    
  plt.show()

In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기 : ', data_batch.shape)
    print('라벨 데이터 크기 : ', labels_batch.shape)
    break

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch = 100, epochs = 30, validation_data = validation_generator, validation_steps = 50)


In [ ]:
model.save('../working/cats_and_dogs_small_1.h5')  # 학습 결과저장

In [ ]:
from keras.models import load_model
model = load_model('../working/cats_and_dogs_small_1.h5')
validation_generator.class_indices
model.predict_generator(validation_generator, steps = 5)

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape=(150,150,3))
conv_base.summary()

In [ ]:

conv_base1 = ResNet101(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
conv_base1.summary()

In [ ]:

samples = ['The cat sat on the mat,','The dog ate my homework']
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 10
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i , sample in enumerate(samples):                                                   #문장의번호 
    for j , word in list(enumerate(sample.split()))[:max_length]:            #문장내에서 단어의 위치
        index = token_index.get(word)
        results[i,j,index] = 1
print(token_index)
print(results)

one hot encoding 구현

In [ ]:

samples = ['The cat sat on the mat,','The dog ate my homework']
tokenizer = Tokenizer(num_words = 10)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode = 'binary')
word_index = tokenizer.word_index
print('Found %s unique tokens.' %len(word_index))
print(word_index)
print(sequences)
print(one_hot_results)

one hot encoding using keras API

In [ ]:


max_features = 10000
maxlen = 20

(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words = max_features)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

IMDB Data Set을 이용하여 문자열 처리 (영화 리뷰를 통해 영화 평가가 긍정인지 부정인지 확인)

In [ ]:


model = Sequential()
model.add(Embedding(10000,8,input_length = maxlen))

model.add(Flatten())

model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_split = 0.2)

In [ ]:
pre = model .predict(x_test)
pre

In [ ]:


acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
y_train

In [ ]:
imdb_dir = '../input/imdbkeras/aclImdb/aclImdb/'

train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []

for label_type in['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding = 'utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

Embedding 방식을 사용하여 각 단어간 관계를 파악하여 결과 도출

In [ ]:


maxlen = 100
training_samples = 10000
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' %len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)    #빈공간은 0으로 채움

labels = np.asarray(labels)                       #List to np
print('데이터 텐서의 크기:', data.shape)
print('라벨의 크기:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples : training_samples + validation_samples]
y_val = labels[training_samples : training_samples + validation_samples]

In [ ]:
glove_dir = '../input/imdbkeras/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = 'utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print('%s개의 단어 벡터를 찾았습니다.'%len(embeddings_index))

In [ ]:
embedding_dim = 100
embedding_matrix= np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i<max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data = (x_val, y_val))
model.save_weights('../working/pre_trainded_glove_model.h5')

In [ ]:
x_test = x_val[:10]
y_test = y_val[:10]
print(y_test)
pre = model.predict(x_test)
print(pre)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Embedding 방식보다 그냥 one hot encoding 방식만 적용했을 경우 정확도가 더 높다.

In [ ]:
from keras.datasets import reuters
(train_data, train_label), (test_Data, test_labels) = reuters.load_data(num_words = 10000)
word_index = reuters.get_word_index()
reverse_word_index = idct([(value,key)for (key,value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[0]])


In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = idct([(value,key)for (key,value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[0]])

In [ ]:
def vectorize_sequences(sequences, dimentsion):
    results = np.zeros((len(sequence)))
    for i , sequence in enumerate(sequences):
        result[i,sequence] =1,
    
    x_test = vectorsize =  

In [ ]:
train_dir_path = '../input/furniture/img/train/'
validation_dir_path = '../input/furniture/img/val/'
validation_dir_path = '../input/furniture/img/val/'


firniture train and validation directory path

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3, 3), activation = 'relu', input_shape = (150,150,3)))

model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(5, activation = 'softmax'))
model.summary()


model 구축

In [ ]:
model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_dir_path, target_size = (150,150), batch_size = 20, class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir_path, target_size = (150,150), batch_size = 20, class_mode = 'categorical')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기 : ', data_batch.shape)
    print('라벨 데이터 크기 : ', labels_batch.shape)
    break
    

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch = 40, epochs = 30, validation_data = validation_generator, validation_steps = 5)


In [ ]:
model.save('../working/furniture_detect.h5')

In [ ]:
from keras.models import load_model
model = load_model('../working/furniture_detect.h5')
validation_generator.class_indices
model.predict_generator(validation_generator, steps = 5)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
pre = model.predict_generator(validation_generator, steps = 1)
for p in pre :
    print(np.argmax(p))

In [ ]:
for i in range(num):
    plt.figure(i)
    imgplot = plt.imshow(validation_generator[0][0][i])
    x = validation_generator[0][0][i]
    x = x.reshape(1,150,150,3)
    pre = model.predict(x)
    print('예측 : ',np.argmax(pre), end = ' / ')
    print('정답 : ',validation_generator[0][1][i])
plt.show()

validation data를 사용하여 평가

In [ ]:
model = Sequential()
model.add(Embedding(10000,32))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = False))  #default = False
model.summary()

RNN(using SimpleRNN)의 기본 구조


In [ ]:
max_features = 10000
maxlen = 500
batch_size = 32

print('data loading..')
(input_train, y_train),(input_test, y_test) = imdb.load_data(num_words = max_features)
print(len(input_train), '훈련 시퀀스')
print(len(input_test), '테스트 시퀀스')

print('시퀀스 패딩 (samples x time)')
input_train = pad_sequences(input_train, maxlen = maxlen)
input_test = pad_sequences(input_test, maxlen = maxlen)

print('input_train 크기 : ', input_train.shape)
print('input_test 크기 : ', input_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32, return_sequences = True))
model.add(SimpleRNN(32, return_sequences = False))  #default = False
model.add(Dense(1, activation = 'tanh'))
model.summary()

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,epochs = 10, batch_size = 128, validation_split = 0.2)

In [ ]:
model.save('../working/rnnTest.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32, return_sequences = False))
model.add(Dense(1, activation = 'tanh'))

In [ ]:

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics= ['acc'])
history = model.fit(input_train, y_train, epochs = 10, batch_size = 128, validation_split = 0.2)

In [ ]:
model.save('../working/LSTMTest.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32, return_sequences = True))
model.add(LSTM(32, return_sequences = False))
model.add(Dense(1, activation = 'tanh'))

In [ ]:

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics= ['acc'])
history = model.fit(input_train, y_train, epochs = 10, batch_size = 128, validation_split = 0.2)

In [ ]:
model.save('../working/doubleLSTMTest.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc , 'b', label = 'Validation acc')
plt.title('Trainig and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss , 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:

class BayesianFilter:
    """ 베이지안 필터 """
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
    # 형태소 분석하기 --- (※1)
    def split(self, text):
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results
    # 단어와 카테고리의 출현 횟수 세기 --- (※2)
    def inc_word(self, word, category):
        # 단어를 카테고리에 추가하기
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
    def inc_category(self, category):
        # 카테고리 계산하기
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
    
    # 텍스트 학습하기 --- (※3)
    def fit(self, text, category):
        """ 텍스트 학습 """
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
    
    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
        
    # 카테고리 내부의 단어 출현 비율 계산 --- (※6)
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d